In [54]:

import sys
import os
import glob
import numpy as np
import nibabel as nib
from sklearn.linear_model import LinearRegression
import warnings


#subject = 'PNC001'
#funcDir = '/data_/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC001/ses-01/func/desc-me_task-rest_bold/'
#outDir = '/data_/mica1/03_projects/hans/7T/func/'
#func_lab = 'sub-PNC001_ses-01_space-func_desc-me_mainPhaseScan'

#labelDir = 'mainPaseScan'
#parcDir = sys.argv[4]
#volmDir = sys.argv[5]
#performNSR = sys.argv[6]
#performGSR = sys.argv[7]
#noFC = sys.argv[9]

In [55]:
# Load confound files
#os.chdir(funcDir+'/volumetric/')
#x_spike = " ".join(glob.glob(funcDir+'/volumetric/'+'*spikeRegressors_FD.1D'))


#x_dof = " ".join(glob.glob(funcDir+'/volumetric/*'+func_lab+'.1D'))
# x_refmse = " ".join(glob.glob(funcDir+'/volumetric/'+'*metric_REFMSE.1D'))
#x_fd = " ".join(glob.glob(funcDir+'/volumetric/'+'*metric_FD*'))
#x_csf = " ".join(glob.glob(funcDir+'/volumetric/'+'*CSF*'))
#x_wm = " ".join(glob.glob(funcDir+'/volumetric/'+'*WM*'))
#x_gs = " ".join(glob.glob(funcDir+'/volumetric/'+'*global*'))

In [56]:
def expand_dim(Data):
    if Data.ndim == 1:
        Data = np.expand_dims(Data, axis=1)
    return Data

def get_regressed_data(x_spike, Data, performNSR, performGSR, Data_name):
    """ Nuisance signal regression: spikes and (optional) WM/CSF:
        This function loads and generates the regression matrix according to the input parameters
        and returns the data corrected by the selected regresors (wm, csf, gs).
        By default will regress motion parameters and spikeRegressors
    Parameters
    ----------
    x_spike : str (spikeRegressors_FD file)
    Data : array
    performNSR : str (0,1)
    performGSR : str (0,1)
    Return
    ------
    Data_corr : array of data corrected
    """
    #dof = np.loadtxt(x_dof)
    #csf = expand_dim(np.loadtxt(x_csf))
    wm = expand_dim(np.loadtxt(x_wm))
    #gs = expand_dim(np.loadtxt(x_gs))
    spike = []
    print('')
    def check_arrays():
        if np.array_equal(mdl, ones) != True:
            print('apply regression')
            slm = LinearRegression().fit(mdl, Data)
            Data_res = Data - np.dot(mdl, slm.coef_.T)
        else:
            Data_res = Data
        return Data_res
    if x_spike:
        spike = expand_dim(np.loadtxt(x_spike))
        print(spike.shape)
        ones = np.ones((spike.shape[0], 1))
        mdl = []

        # regress out spikes from individual timeseries
        if performNSR == "1":
            print(Data_name + ' model : func ~ spikes + dof + wm + csf')
            mdl = np.append(np.append(np.append(np.append(ones, spike, axis=1), dof, axis=1), wm, axis=1), csf, axis=1)
        elif performGSR == "1":
            print(Data_name + ' model : func ~ spikes + dof + wm + csf + gs')
            mdl = np.append(np.append(np.append(np.append(np.append(ones, spike, axis=1), dof, axis=1), wm, axis=1), csf, axis=1), gs, axis=1)
        else:
            print(Data_name + 'Default model : func ~ spikes')
            mdl = np.append(ones, spike, axis=1)
        # apply regression
        Data_corr = check_arrays()
    else:
        ones = np.ones((wm.shape[0], 1))
        print('NO spikeRegressors_FD file, will skip loading: ' + Data_name)
        if performNSR == 1:
            print(Data_name + ', model : func ~ dof + wm + csf')
            mdl = np.append(np.append(np.append(ones, dof, axis=1), wm, axis=1), csf, axis=1)
        elif performGSR == 1:
            print(Data_name + ', model : func ~ dof + wm + csf + gs')
            mdl = np.append(np.append(np.append(np.append(ones, dof, axis=1), wm, axis=1), csf, axis=1), gs, axis = 1)
        else:
            print(Data_name + ', model : none')
            mdl = ones
        # apply regression
        Data_corr = check_arrays()
    return Data_corr

In [57]:
for i in [1,4,5,6,7,9,10,11,12,13]:
    if i<10:
        subject = str('PNC00'+str(i))
    else:
        subject = str('PNC0'+str(i))
    funcDir = str('/data_/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-'+subject+'/ses-01/func/desc-me_task-rest_bold/')
    outDir = '/host/percy/local_raid/hans/amyg/func/'
    func_lab = str('sub-'+subject+'_ses-01_space-func_desc-me_mainPhaseScan')
    # Load confound files
    os.chdir(funcDir+'/volumetric/')
    x_spike = " ".join(glob.glob(funcDir+'/volumetric/'+'*spikeRegressors_FD.1D'))
    x_wm = " ".join(glob.glob(funcDir+'/volumetric/'+'*WM*'))
    performNSR=0
    performGSR=0
    data= np.loadtxt(str(outDir + 'sub-'+subject+'_ses-01_space-func_desc-me_timeseries_amyg.txt'),delimiter = ' ')
    data=np.transpose(data)
    print(data.shape)
    #skip pNC007 because they have perfect timeseries

    data_corr = get_regressed_data(x_spike, data, performNSR, performGSR, 'conte69')
    
    np.savetxt(outDir + subject + '_01_amyg_func_space-func_desc-me_timeseries_clean.txt', data_corr, fmt='%.6f')

(275, 138)

(275, 5)
conte69Default model : func ~ spikes
apply regression
(275, 106)

(275, 24)
conte69Default model : func ~ spikes
apply regression
(275, 138)

(275, 7)
conte69Default model : func ~ spikes
apply regression
(275, 119)

(275, 7)
conte69Default model : func ~ spikes
apply regression
(275, 99)

NO spikeRegressors_FD file, will skip loading: conte69
conte69, model : none
(275, 73)

(275, 13)
conte69Default model : func ~ spikes
apply regression
(275, 123)

(275, 9)
conte69Default model : func ~ spikes
apply regression
(275, 42)

(275, 11)
conte69Default model : func ~ spikes
apply regression
(275, 79)

(275, 19)
conte69Default model : func ~ spikes
apply regression
(272, 58)

(272, 8)
conte69Default model : func ~ spikes
apply regression


In [13]:
#print(data_corr)
#print(data)
#print(data.shape)
#data2=np.loadtxt('
print(data[:,0])
print(data_corr[:,0])

[-1.590004e+02  8.749979e+01  1.975962e+00 -8.318652e+01  1.318563e+02
  7.571494e+01 -4.431759e+01  5.845951e+01 -6.547567e+01 -1.607918e+01
  8.012846e+01 -1.457896e+02  3.761959e+01 -1.330535e+02  6.389583e+01
  1.154240e+02  7.147880e+01 -3.605684e+01  9.001653e+01 -1.084988e+02
  7.785134e+01  1.114383e+02 -5.700290e+01 -8.992695e+01 -1.749871e+02
  2.993674e+01  1.001873e+02 -1.130952e+02  2.847246e+01 -6.395670e+01
  3.426270e+00  7.462568e+01  8.147819e+01  2.852704e+01  1.933417e+01
 -7.613296e+00  6.549015e+01  6.564488e+00  9.227464e+01 -2.897904e+01
 -8.285865e+01 -3.797813e+00  4.037171e+01 -3.472364e+01  1.517039e+02
 -1.951858e+02 -1.516405e+00 -1.367762e+02  2.107060e+01 -9.450814e+01
  9.519571e+00  6.213581e+01 -2.255532e+01 -1.747440e+00 -9.482340e-01
 -1.686019e+02 -1.584726e+02  5.040864e+01  6.265765e+01 -1.898389e+01
  7.022623e+00  5.443355e+01  5.347701e+01 -3.568598e+00  9.730860e+01
  9.346241e+01  2.294611e+02 -2.829239e+01  7.266077e+01 -1.529589e+01
  2.79

In [14]:
import nibabel as nib
sub='PNC013'
ses='01'
funcNif = nib.load(outDir+sub+'_'+ses+'_amyg_mask_funcSpace_1sd_test.nii.gz')
funcMask = np.array(funcNif.dataobj)
#coords
num=0
for i in range(len(funcMask[:,1,1])):
    for j in range(len(funcMask[1,:,1])):
        for k in range(len(funcMask[1,1,:])):
            if funcMask[i,j,k] != 0:
                num=num+1
print(num)

50


In [7]:
str('/data_/mica1/03_projects/hans/7T/outputs/amyg_mask_'+sub+'_'+ses+'_left_roi.nii.gz')
sub='PNC001'
ses='01'
funcNif = nib.load(str('/data_/mica1/03_projects/hans/7T/outputs/amyg_mask_'+sub+'_'+ses+'_left_roi.nii.gz'))
funcMask = np.array(funcNif.dataobj)
#coords
num=0
for i in range(len(funcMask[:,1,1])):
    for j in range(len(funcMask[1,:,1])):
        for k in range(len(funcMask[1,1,:])):
            if funcMask[i,j,k] != 0:
                num=num+1
print(num)

10102
